<a href="https://colab.research.google.com/github/hammad93/hurricane-tts/blob/main/hurricane_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Please make sure to run all tests.

In [1]:
%cd ~/
!git clone https://github.com/hammad93/hurricane-tts.git

/root
Cloning into 'hurricane-tts'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 277 (delta 37), reused 5 (delta 1), pack-reused 195
Receiving objects: 100% (277/277), 1.29 MiB | 12.86 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [2]:
!pip install -r hurricane-tts/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.5/220.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [46]:
import os
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html#environment-variables
vars = ["AZURE_OPENAI_API_KEY", "AZURE_REDIS_KEY", "AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY"]
if None in [os.getenv(env) for env in vars] :
  from google.colab import userdata
  for var in vars :
    os.environ[var] = userdata.get(var)

In [4]:
%cd ~/hurricane-tts
!python ~/hurricane-tts/test.py

/root/hurricane-tts
.Here are the constructed messages: [{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'test'}]
..
----------------------------------------------------------------------
Ran 3 tests in 8.910s

OK


In [5]:
import prompts
import utils
import db
# we generate prompts by ingesting live hurricane data and supported languages
storm_data = utils.transform_storm_data()
prompt_data = prompts.generate_prompts()

# metadata related to multilingual generation
supported_langs = prompts.unique_lang_list()
supported_langs_df = utils.tts_langs()

In [6]:
# example
print(prompt_data['storms'][0])

Provided are the tropical storm records with geographic coordinates and wind speed in knots.
Please respond with at least 2 of the supported languages by utilizing language geography and the storm's latitude and longitude.
The response must be a Python list meant to be input as a string into eval(). Please make sure the languages are in the supported list.
As an example, ["Chinese, Mandarin", "Haitian Creole", "Indonesian"]
[{'type': 'history', 'lat': 10.6, 'lon': -114.4, 'time': '2023-11-17 18:00:00', 'wind_speed': 25}, {'type': 'history', 'lat': 10.5, 'lon': -114.7, 'time': '2023-11-18 00:00:00', 'wind_speed': 25}, {'type': 'history', 'lat': 10.4, 'lon': -115.0, 'time': '2023-11-18 06:00:00', 'wind_speed': 25}, {'type': 'history', 'lat': 10.3, 'lon': -115.4, 'time': '2023-11-18 12:00:00', 'wind_speed': 25}, {'type': 'history', 'lat': 10.2, 'lon': -115.8, 'time': '2023-11-18 18:00:00', 'wind_speed': 25}, {'type': 'history', 'lat': 10.1, 'lon': -116.2, 'time': '2023-11-19 00:00:00', 'w

# Language Geography inference from tropical storm geographical coorindates
We're trying to answer the question, "Which languages do we produce speech for to report on this tropical storm?" This pipeline will produce an English output but also utilize massively multilingual capabilities such that emergency notices are also in local languages

In [7]:
storm_langs = {} # keys are storm id's and values are the languages
storm_chats = {} # stores chat histories for storms
for index, storm_prompt in enumerate(prompt_data['storms']) :
  storm_id = list(storm_data.keys())[index]
  # Get the languages to generate the report for this storm
  retries = 20 # sometimes it fails, so we retry it
  while retries > 0 :
    response = prompts.chat(system=prompt_data['system'], message=storm_prompt)
    result = utils.llm_response_transform(
        resp=response,
        supported_langs=supported_langs)
    if result :
      print(f'Successfully parsed {response}')
      break
    else :
      retries = retries - 1
      print(f"Failed. Retries left: {retries}")
  if retries < 1 :
    raise Exception("Couldn't produce a correct output from LLM.")

  # store results
  storm_langs[storm_id] = {'names': ['English'] + result}
  storm_chats[storm_id] = {
      'history' : [{'role': 'system', 'content': prompt_data['system']},
                   {'role': 'user', 'content': storm_prompt},
                   {'role': 'assistant', 'content': response}]
  }

Here are the constructed messages: [{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nThis is the list of supported languages,\n{'BorÃ´ro', 'Crimean Tatar', 'Tairora, South', 'Nomaande', 'Yamdena', 'Mazatec, IxcatlÃ¡n', 'Tacana', 'Tunebo, Central', 'Konkomba', 'Inga', 'Pahari, Kullu', 'PaumarÃ\\xad', 'Semelai', 'Zapotec, Yareni', 'Vidunda', 'Lango', 'Ayta, Mag-antsi', 'Laz', 'Wapishana', 'Datooga', 'Somali', 'Yalunka', 'Chipaya', 'Chontal, Tabasco', 'Ifugao, Amganad', 'Kako', 'Ndogo', 'Kafa', 'Prai', 'Hindi', 'Quechua, North JunÃ\\xadn', 'Chinantec, Lalana', 'Karachay-Balkar', 'Odia', 'Chinese, Hakka', 'Khmu', 'Gela', 'Bahnar', 'Dzongkha', 'Erzya', 'Poqomchiâ\\x80\\x99', 'Hindustani, Sarnami', 'Holiya', 'Kutep', 'Kâ\\x80\\x99icheâ\\x80\\x99', 'Adele', 'Chol', 'Tigrigna', 'Uyghur', 'Puinave', 'Mofu-Gudur', 'Wali', 'Awa', 'Hindi, Fiji', 'Amis', 'Desiya', 'Kisar', 'Mixe, CoatlÃ¡n', 'Zapotec, Cajonos', 'Cebuano', 'Ahanta', 'Guahibo', '

# Tropical Storm Report
This code will create the report in the language specified based on the tropical storm.

In [8]:
with open('prompts/report-prompt.txt', 'r') as file:
  report_prompt = file.read()
for storm in storm_langs:
  print(storm)
  reports = []
  for lang in storm_langs[storm]['names']:
    print(storm_chats[storm]['history'])
    # Construct the prompt
    message = report_prompt.format(lang=lang, id=storm[2:-4])
    print(message)
    # Send to LLM
    response = prompts.chat(
        message = message, history = storm_chats[storm]['history'])
    print(response)
    # store data
    reports.append(response)
  storm_langs[storm]['reports'] = reports

EP942023
[{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nThis is the list of supported languages,\n{'BorÃ´ro', 'Crimean Tatar', 'Tairora, South', 'Nomaande', 'Yamdena', 'Mazatec, IxcatlÃ¡n', 'Tacana', 'Tunebo, Central', 'Konkomba', 'Inga', 'Pahari, Kullu', 'PaumarÃ\\xad', 'Semelai', 'Zapotec, Yareni', 'Vidunda', 'Lango', 'Ayta, Mag-antsi', 'Laz', 'Wapishana', 'Datooga', 'Somali', 'Yalunka', 'Chipaya', 'Chontal, Tabasco', 'Ifugao, Amganad', 'Kako', 'Ndogo', 'Kafa', 'Prai', 'Hindi', 'Quechua, North JunÃ\\xadn', 'Chinantec, Lalana', 'Karachay-Balkar', 'Odia', 'Chinese, Hakka', 'Khmu', 'Gela', 'Bahnar', 'Dzongkha', 'Erzya', 'Poqomchiâ\\x80\\x99', 'Hindustani, Sarnami', 'Holiya', 'Kutep', 'Kâ\\x80\\x99icheâ\\x80\\x99', 'Adele', 'Chol', 'Tigrigna', 'Uyghur', 'Puinave', 'Mofu-Gudur', 'Wali', 'Awa', 'Hindi, Fiji', 'Amis', 'Desiya', 'Kisar', 'Mixe, CoatlÃ¡n', 'Zapotec, Cajonos', 'Cebuano', 'Ahanta', 'Guahibo', 'Minangkabau', 'Anyin', 'Ar

In [9]:
storm_langs

{'EP942023': {'names': ['English', 'Spanish', 'Tagalog'],
  'reports': ['"In recent history, there have been many tropical storms that have caused significant damage and loss of life. As we track the path of storm number 94, it is important that we take necessary precautions to protect ourselves and our loved ones. We urge everyone in the affected areas to stay informed and follow all safety guidelines. Let us come together as a community to weather this storm and emerge stronger on the other side."',
   'En los últimos años, hemos experimentado clima extremo debido a los efectos del cambio climático. En este momento nos enfrentamos a la tormenta número 94, que está actualmente activa y tiene el potencial de causar daños significativos. Es importante mantenerse alerta y preparado para cualquier eventualidad. Si se encuentra en una zona propensa a tormentas, asegúrese de tener almacenados suministros de emergencia, incluyendo agua potable y alimentos no perecederos. También es important

The following are from the TTS implementation
https://github.com/facebookresearch/fairseq/blob/main/examples/mms/tts/tutorial/MMS_TTS_Inference_Colab.ipynb


In [10]:
%cd ~/
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install "numpy<1.24"
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd

/root
Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (81/81), 3.33 MiB | 18.06 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/root/vits
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.8 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201367 sha256=878eff28ab668d3f0c4b63176d04d6c6b1c2cc8d5a4f5486390c9741f90cced7
  Stored in directory: /root/.cache/pip/wheels/bf/b7/85/2f8044306ccec014930aea23ad4852fca9e2584e21c6972bc6
Successfully built librosa
  Attempting uninstall: librosa
  

'/root/vits'

In [11]:
# we have the data we need and the utils class is also used in the next code
import sys
del sys.modules["utils"]

In [34]:
%cd ~/hurricane-tts/
import fair_mms_tts
%cd ~/vits
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn

/root/hurricane-tts
/root/vits


In [13]:
# match language names to iso code
for storm in storm_langs :
  langs = storm_langs[storm]['names']
  print(langs)
  codes = [supported_langs_df[
      supported_langs_df['Language Name'] == name
  ]['ISO 693-3'].iloc[0] for name in langs]
  print(codes)
  storm_langs[storm]['codes'] = codes

['English', 'Spanish', 'Tagalog']
['eng', 'spa', 'tgl']
['English', 'Spanish', 'Quechua, Huaylas Ancash']
['eng', 'spa', 'qwh']


In [14]:
import soundfile as sf
def generate_audio(report, lang_code, out):
  '''
  Generates the audio given the report and lang_code
  '''
  ckpt_dir = fair_mms_tts.download(lang_code)
  import torch
  if torch.cuda.is_available():
      device = torch.device("cuda")
  else:
      device = torch.device("cpu")
  print(device)
  print(f"Run inference with {device}")
  vocab_file = f"{ckpt_dir}/vocab.txt"
  config_file = f"{ckpt_dir}/config.json"
  assert os.path.isfile(config_file), f"{config_file} doesn't exist"
  hps = utils.get_hparams_from_file(config_file)

  text_mapper = fair_mms_tts.TextMapper(vocab_file)
  net_g = SynthesizerTrn(
      len(text_mapper.symbols),
      hps.data.filter_length // 2 + 1,
      hps.train.segment_size // hps.data.hop_length,
      **hps.model)
  net_g.to(device)
  _ = net_g.eval()

  g_pth = f"{ckpt_dir}/G_100000.pth"
  print(f"load {g_pth}")

  _ = utils.load_checkpoint(g_pth, net_g, None)
  txt = fair_mms_tts.preprocess_text(report, text_mapper, hps, lang=lang_code)
  stn_tst = text_mapper.get_text(txt, hps)

  with torch.no_grad():
      x_tst = stn_tst.unsqueeze(0).to(device)
      x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
      hyp = net_g.infer(
          x_tst, x_tst_lengths, noise_scale=.667,
          noise_scale_w=0.8, length_scale=1.0
      )[0][0,0].cpu().float().numpy()

  print(f"Generated audio")
  sf.write(out, hyp, hps.data.sampling_rate)

In [15]:
# generate audio
import time
time_fname = int(time.time())
outputs = []
outputs_metadata = {}
for storm in storm_langs :
  print(storm)
  current = storm_langs[storm]
  for index, code in enumerate(current['codes']):
    path = f"/root/{storm}_{time_fname}_{current['names'][index]}.wav"
    prompt = current['reports'][index]
    outputs.append(path)
    outputs_metadata[path] = {
        'prompt': prompt,
        'lang': code
    }
    generate_audio(report = prompt,
                   lang_code = code,
                   out = path)

EP942023
Download model for language: eng
Model checkpoints in ./eng: ['vocab.txt', 'config.json', 'G_100000.pth']
cuda
Run inference with cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


load ./eng/G_100000.pth
eng
text after filtering OOV: in recent history there have been many tropical storms that have caused significant damage and loss of life as we track the path of storm number 4 it is important that we take necessary precautions to protect ourselves and our loved ones we urge everyone in the affected areas to stay informed and follow all safety guidelines let us come together as a community to weather this storm and emerge stronger on the other side
Generated audio
Download model for language: spa
Model checkpoints in ./spa: ['vocab.txt', 'config.json', 'G_100000.pth']
cuda
Run inference with cuda
load ./spa/G_100000.pth
spa
text after filtering OOV: en los últimos años hemos experimentado clima extremo debido a los efectos del cambio climático en este momento nos enfrentamos a la tormenta número 94 que está actualmente activa y tiene el potencial de causar daños significativos es importante mantenerse alerta y preparado para cualquier eventualidad si se encuentr

In [47]:
%cd ~/hurricane-tts/
import db
import json
print(outputs)
db.upload_latest_audios(outputs, outputs_metadata)

/root/hurricane-tts
['/root/EP942023_1700604629_English.wav', '/root/EP942023_1700604629_Spanish.wav', '/root/EP942023_1700604629_Tagalog.wav', '/root/AL992023_1700604629_English.wav', '/root/AL992023_1700604629_Spanish.wav', '/root/AL992023_1700604629_Quechua, Huaylas Ancash.wav']
'NoneType' object has no attribute 'content'
'NoneType' object has no attribute 'content'
'NoneType' object has no attribute 'content'
'NoneType' object has no attribute 'content'
'NoneType' object has no attribute 'content'
'NoneType' object has no attribute 'content'


In [28]:
#!git -C /root/hurricane-tts/ pull

Already up to date.


In [32]:
# reload a class, can take a while to figure out so don't delete this cell
#import sys
#del sys.modules["db"]